In [57]:
import numpy as np
import pandas as pd
import requests
import re
import time
import pickle

from bs4 import BeautifulSoup as bs


In [5]:
#네이버 도전웹툰 1~1000page의 모든 웹툰 제목(title), 링크(link), 평점(star)를 수집
def get_challenge_webtoon(page):
    url = 'https://comic.naver.com/genre/challenge.nhn?m=list&page={}'.format(page)
    response = requests.get(url)
    webtoons = bs(response.content, 'html.parser')
    
    domain = 'https://comic.naver.com'
    for webtoon in webtoons.select('.challengeInfo'):
        table.append({
            
            'title' : webtoon.select('a')[0].text.strip(),
            'link' : domain + webtoon.select('a')[0].get('href'),
            'star' : webtoon.select_one('.star').get('title')
        
        })

#link를 받아 에피소드의 개수를 체크함 (첫번째 url = 마지막 에피소드 url에 회차정보가 담겨 있음)       
def check_num_episode(title_link):
    response = requests.get(title_link)
    info = bs(response.content, 'html.parser').select_one('.title > a').get('href')
    return info[-4:]

In [9]:
table = []
for page in range(1,5):
    get_challenge_webtoon(page)

result = pd.DataFrame(table)
result.star = result.star.apply(lambda x: float(x.replace('점','')))
result['title_id'] = result.link.apply(lambda x:x[-6:])
result.tail()

,link,star,title
91,https://comic.naver.com/challenge/list.nhn?tit...,7.75,엄마는 오늘도
92,https://comic.naver.com/challenge/list.nhn?tit...,9.14,지박스쿨
93,https://comic.naver.com/challenge/list.nhn?tit...,9.00,웃는 방범
94,https://comic.naver.com/challenge/list.nhn?tit...,10.00,넷웍마의 추억
95,https://comic.naver.com/challenge/list.nhn?tit...,10.00,그땐-그랬지


In [11]:
num = []
links = result.link

for i in range(len(links)):
    try:
        num.append(check_num_episode(links[i]))
    except AttributeError:
        num.append('Error')
    
    print(str(i+1) +'/{}'.format(len(links)))

result['num_episode'] = num
result['num_episode'] = result.num_episode.apply(lambda x: re.sub('[a-z=]','',x))

0/96
1/96
2/96
3/96
4/96
5/96
6/96
7/96
8/96
9/96
10/96
11/96
12/96
13/96
14/96
15/96
16/96
17/96
18/96
19/96
20/96
21/96
22/96
23/96
24/96
25/96
26/96
27/96
28/96
29/96
30/96
31/96
32/96
33/96
34/96
35/96
36/96
37/96
38/96
39/96
40/96
41/96
42/96
43/96
44/96
45/96
46/96
47/96
48/96
49/96
50/96
51/96
52/96
53/96
54/96
55/96
56/96
57/96
58/96
59/96
60/96
61/96
62/96
63/96
64/96
65/96
66/96
67/96
68/96
69/96
70/96
71/96
72/96
73/96
74/96
75/96
76/96
77/96
78/96
79/96
80/96
81/96
82/96
83/96
84/96
85/96
86/96
87/96
88/96
89/96
90/96
91/96
92/96
93/96
94/96
95/96


In [55]:
result.num_episode.max()

'98'

In [89]:
url = 'https://comic.naver.com/genre/challenge.nhn?m=list&page=1'
domain = 'https://comic.naver.com'
response = requests.get(url)
webtoons = bs(response.content, 'html.parser')

In [90]:
malsook = webtoons.select('.challengeInfo')[0]

In [91]:
title_link = domain + malsook.select('a')[0].get('href')

In [92]:
response = requests.get(title_link)
html = bs(response.content, 'html.parser')

In [93]:
html.select('td.title')[0].select_one('a').get('href')

'/challenge/detail.nhn?titleId=742474&no=1'

In [95]:
malsook

<div class="challengeInfo">
<h6 class="challengeTitle">
<a href="/challenge/list.nhn?titleId=742474" onclick="nclk_v2(event,'acl.tit','','1')">
                                    뽑기 수호자
                                </a>
</h6>
<a class="user" href="#" onclick="nclk_v2(event,'acl.writer','','1'); return artistAction.viewArtist('742474', this);">
                                kbj_0403
                            </a>
<div class="summary">군대 판타지 만화!</div>
<div class="rating_type">
<span class="star" title="0.00점"><em style="width:0.00%">평점</em></span>
<strong>0.00</strong>
</div>
</div>